In [1]:
from __future__ import division
import argparse

from PIL import Image
import numpy as np
import gym

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Input, Concatenate
from keras.optimizers import Adam
import keras.backend as K

from rl.agents.ddpg import DDPGAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
from rl.random import OrnsteinUhlenbeckProcess

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt
import sys
from gym_unity.envs import UnityEnv

%matplotlib inline

print("Python version:")
print(sys.version)

# check Python version
if (sys.version_info[0] < 3):
    raise Exception("ERROR: ML-Agents Toolkit (v0.3 onwards) requires Python 3")


Python version:
3.6.7 |Anaconda, Inc.| (default, Oct 23 2018, 14:01:38) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [3]:
class BallVecProcessor(Processor):
    def process_action(self, action):
        #print(action)
        return action
    def process_info(self, info):
        key, value = info.items()
        #print(key)
        #print(value)
        #print(value[1].rewards)
        key = 1
        value = value[1].rewards
        info = {key: value}
        return info
    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)    

In [4]:
env_name = "mlagents/envs/3DBall_128"  # Name of the Unity environment binary to launch
env = UnityEnv(env_name, worker_id=0, use_visual=False)

nb_actions = 2
print(str(env))

INFO:mlagents.envs:
'Ball3DAcademy' started successfully!
Unity Academy name: Ball3DAcademy
        Number of Brains: 1
        Number of External Brains : 1
        Reset Parameters :
		
Unity brain name: Ball3DBrain
        Number of Visual Observations (per agent): 1
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): [2]
        Vector Action descriptions: , 
INFO:gym_unity:1 agents within environment.


<UnityEnv instance>


In [5]:
actor = Sequential()
actor.add(Flatten(input_shape=(1,) + env.observation_space.shape))
actor.add(Dense(16))
actor.add(Activation('relu'))
actor.add(Dense(16))
actor.add(Activation('relu'))
actor.add(Dense(16))
actor.add(Activation('relu'))
actor.add(Dense(nb_actions))
actor.add(Activation('linear'))
print(actor.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                144       
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_3 (Activation)    (None, 16)                0         
__________

In [6]:
action_input = Input(shape=(nb_actions,), name='action_input')
observation_input = Input(shape=(1,) + env.observation_space.shape, name='observation_input')
flattened_observation = Flatten()(observation_input)
x = Concatenate()([action_input, flattened_observation])
x = Dense(32)(x)
x = Activation('relu')(x)
x = Dense(32)(x)
x = Activation('relu')(x)
x = Dense(32)(x)
x = Activation('relu')(x)
x = Dense(1)(x)
x = Activation('linear')(x)
critic = Model(inputs=[action_input, observation_input], outputs=x)
print(critic.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
observation_input (InputLayer)  (None, 1, 8)         0                                            
__________________________________________________________________________________________________
action_input (InputLayer)       (None, 2)            0                                            
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 8)            0           observation_input[0][0]          
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 10)           0           action_input[0][0]               
                                                                 flatten_2[0][0]                  
__________

In [7]:
processor = BallVecProcessor()
memory = SequentialMemory(limit=100000, window_length=1)
random_process = OrnsteinUhlenbeckProcess(size=nb_actions, theta=.15, mu=0., sigma=.3)
agent = DDPGAgent(nb_actions=nb_actions, actor=actor, critic=critic, critic_action_input=action_input,
                  memory=memory, nb_steps_warmup_critic=100, nb_steps_warmup_actor=100,
                  random_process=random_process, gamma=.99, target_model_update=1e-3, processor = processor)
agent.compile(Adam(lr=.001, clipnorm=1.), metrics=['mae'])

In [8]:
agent.fit(env, nb_steps=50000, visualize=True, verbose=2)



Training for 50000 steps ...
    19/50000: episode: 1, duration: 0.563s, episode steps: 19, steps per second: 34, episode reward: 0.800, mean reward: 0.042 [-1.000, 0.100], mean action: -0.074 [-0.965, 0.706], mean observation: 0.189 [-6.867, 4.000], loss: --, mean_absolute_error: --, mean_q: --
    36/50000: episode: 2, duration: 0.172s, episode steps: 17, steps per second: 99, episode reward: 0.600, mean reward: 0.035 [-1.000, 0.100], mean action: -0.085 [-1.028, 0.495], mean observation: 0.120 [-7.848, 4.000], loss: --, mean_absolute_error: --, mean_q: --
    56/50000: episode: 3, duration: 0.198s, episode steps: 20, steps per second: 101, episode reward: 0.900, mean reward: 0.045 [-1.000, 0.100], mean action: -0.068 [-0.960, 0.323], mean observation: 0.153 [-7.848, 4.000], loss: --, mean_absolute_error: --, mean_q: --
    72/50000: episode: 4, duration: 0.162s, episode steps: 16, steps per second: 99, episode reward: 0.500, mean reward: 0.031 [-1.000, 0.100], mean action: -0.102 [-

   551/50000: episode: 30, duration: 0.318s, episode steps: 19, steps per second: 60, episode reward: 0.800, mean reward: 0.042 [-1.000, 0.100], mean action: -0.028 [-1.821, 1.601], mean observation: 0.356 [-7.848, 4.000], loss: 0.006909, mean_absolute_error: 0.071849, mean_q: -0.291069
   570/50000: episode: 31, duration: 0.240s, episode steps: 19, steps per second: 79, episode reward: 0.800, mean reward: 0.042 [-1.000, 0.100], mean action: 0.193 [-1.509, 3.074], mean observation: 0.262 [-5.886, 4.000], loss: 0.008931, mean_absolute_error: 0.083821, mean_q: -0.292774
   584/50000: episode: 32, duration: 0.173s, episode steps: 14, steps per second: 81, episode reward: 0.300, mean reward: 0.021 [-1.000, 0.100], mean action: 0.221 [-1.217, 1.842], mean observation: 0.152 [-8.829, 4.000], loss: 0.006739, mean_absolute_error: 0.074450, mean_q: -0.305965
   604/50000: episode: 33, duration: 0.303s, episode steps: 20, steps per second: 66, episode reward: 0.900, mean reward: 0.045 [-1.000, 0

  1072/50000: episode: 59, duration: 0.303s, episode steps: 23, steps per second: 76, episode reward: 1.200, mean reward: 0.052 [-1.000, 0.100], mean action: 0.063 [-1.594, 4.268], mean observation: 0.407 [-4.905, 4.000], loss: 0.004265, mean_absolute_error: 0.057198, mean_q: -0.242825
  1089/50000: episode: 60, duration: 0.215s, episode steps: 17, steps per second: 79, episode reward: 0.600, mean reward: 0.035 [-1.000, 0.100], mean action: 0.328 [-1.451, 3.833], mean observation: 0.307 [-6.867, 4.000], loss: 0.004426, mean_absolute_error: 0.055560, mean_q: -0.243024
  1106/50000: episode: 61, duration: 0.243s, episode steps: 17, steps per second: 70, episode reward: 0.600, mean reward: 0.035 [-1.000, 0.100], mean action: -0.030 [-0.909, 1.130], mean observation: 0.343 [-7.848, 4.000], loss: 0.004848, mean_absolute_error: 0.059053, mean_q: -0.249043
  1122/50000: episode: 62, duration: 0.215s, episode steps: 16, steps per second: 74, episode reward: 0.500, mean reward: 0.031 [-1.000, 0

  1640/50000: episode: 89, duration: 0.208s, episode steps: 15, steps per second: 72, episode reward: 0.400, mean reward: 0.027 [-1.000, 0.100], mean action: -0.039 [-1.493, 1.771], mean observation: 0.175 [-8.829, 4.000], loss: 0.003033, mean_absolute_error: 0.049859, mean_q: -0.187453
  1654/50000: episode: 90, duration: 0.170s, episode steps: 14, steps per second: 83, episode reward: 0.300, mean reward: 0.021 [-1.000, 0.100], mean action: -0.604 [-3.255, 1.647], mean observation: -0.088 [-8.829, 4.000], loss: 0.003116, mean_absolute_error: 0.045873, mean_q: -0.187394
  1673/50000: episode: 91, duration: 0.246s, episode steps: 19, steps per second: 77, episode reward: 0.800, mean reward: 0.042 [-1.000, 0.100], mean action: -0.452 [-2.306, 0.983], mean observation: 0.386 [-6.867, 4.000], loss: 0.003749, mean_absolute_error: 0.055552, mean_q: -0.211366
  1708/50000: episode: 92, duration: 0.418s, episode steps: 35, steps per second: 84, episode reward: 2.400, mean reward: 0.069 [-1.000

  2162/50000: episode: 118, duration: 0.246s, episode steps: 21, steps per second: 85, episode reward: 1.000, mean reward: 0.048 [-1.000, 0.100], mean action: -0.611 [-4.824, 1.825], mean observation: 0.427 [-6.867, 4.000], loss: 0.001629, mean_absolute_error: 0.038884, mean_q: -0.150627
  2180/50000: episode: 119, duration: 0.282s, episode steps: 18, steps per second: 64, episode reward: 0.700, mean reward: 0.039 [-1.000, 0.100], mean action: -0.259 [-1.440, 1.131], mean observation: 0.297 [-7.848, 4.000], loss: 0.002074, mean_absolute_error: 0.040838, mean_q: -0.167313
  2208/50000: episode: 120, duration: 0.425s, episode steps: 28, steps per second: 66, episode reward: 1.700, mean reward: 0.061 [-1.000, 0.100], mean action: -0.139 [-2.838, 4.244], mean observation: 0.422 [-5.206, 4.000], loss: 0.002217, mean_absolute_error: 0.045703, mean_q: -0.165250
  2223/50000: episode: 121, duration: 0.210s, episode steps: 15, steps per second: 72, episode reward: 0.400, mean reward: 0.027 [-1.

...

 36496/50000: episode: 699, duration: 0.527s, episode steps: 39, steps per second: 74, episode reward: 2.800, mean reward: 0.072 [-1.000, 0.100], mean action: -12.154 [-134.172, 10.128], mean observation: 0.431 [-8.724, 4.000], loss: 0.034680, mean_absolute_error: 0.122147, mean_q: 2.631930
 36523/50000: episode: 700, duration: 0.415s, episode steps: 27, steps per second: 65, episode reward: 1.600, mean reward: 0.059 [-1.000, 0.100], mean action: -17.717 [-92.413, 4.958], mean observation: 0.276 [-6.867, 4.000], loss: 0.030619, mean_absolute_error: 0.113758, mean_q: 2.692477
 36621/50000: episode: 701, duration: 1.323s, episode steps: 98, steps per second: 74, episode reward: 8.700, mean reward: 0.089 [-1.000, 0.100], mean action: -5.296 [-105.164, 16.082], mean observation: 0.235 [-6.867, 4.000], loss: 0.042000, mean_absolute_error: 0.128913, mean_q: 2.641666
 36672/50000: episode: 702, duration: 0.642s, episode steps: 51, steps per second: 79, episode reward: 4.000, mean reward: 0.07

 38260/50000: episode: 728, duration: 0.711s, episode steps: 54, steps per second: 76, episode reward: 4.300, mean reward: 0.080 [-1.000, 0.100], mean action: -8.291 [-89.423, 10.564], mean observation: 0.262 [-6.867, 4.000], loss: 0.028006, mean_absolute_error: 0.111408, mean_q: 2.787579
 38287/50000: episode: 729, duration: 0.350s, episode steps: 27, steps per second: 77, episode reward: 1.600, mean reward: 0.059 [-1.000, 0.100], mean action: -0.109 [-35.068, 30.088], mean observation: -0.670 [-8.704, 4.000], loss: 0.033792, mean_absolute_error: 0.123873, mean_q: 2.736282
 38308/50000: episode: 730, duration: 0.273s, episode steps: 21, steps per second: 77, episode reward: 1.000, mean reward: 0.048 [-1.000, 0.100], mean action: -7.347 [-55.597, 17.902], mean observation: -0.371 [-7.848, 4.000], loss: 0.024022, mean_absolute_error: 0.104003, mean_q: 2.883602
 38332/50000: episode: 731, duration: 0.308s, episode steps: 24, steps per second: 78, episode reward: 1.300, mean reward: 0.054

 39674/50000: episode: 757, duration: 0.296s, episode steps: 22, steps per second: 74, episode reward: 1.100, mean reward: 0.050 [-1.000, 0.100], mean action: -8.887 [-90.316, 7.253], mean observation: 0.113 [-6.867, 4.000], loss: 0.028356, mean_absolute_error: 0.115115, mean_q: 2.835270
 39701/50000: episode: 758, duration: 0.379s, episode steps: 27, steps per second: 71, episode reward: 1.600, mean reward: 0.059 [-1.000, 0.100], mean action: -7.283 [-80.736, 20.031], mean observation: 0.452 [-5.886, 4.000], loss: 0.021650, mean_absolute_error: 0.107723, mean_q: 2.895331
 39756/50000: episode: 759, duration: 0.758s, episode steps: 55, steps per second: 73, episode reward: 4.400, mean reward: 0.080 [-1.000, 0.100], mean action: -7.801 [-154.423, 10.536], mean observation: -0.035 [-6.805, 4.000], loss: 0.038875, mean_absolute_error: 0.123527, mean_q: 2.901737
 39789/50000: episode: 760, duration: 0.472s, episode steps: 33, steps per second: 70, episode reward: 2.200, mean reward: 0.067 

 42627/50000: episode: 786, duration: 8.851s, episode steps: 720, steps per second: 81, episode reward: 70.900, mean reward: 0.098 [-1.000, 0.100], mean action: -0.637 [-90.445, 18.721], mean observation: 0.130 [-6.200, 4.000], loss: 0.027259, mean_absolute_error: 0.106750, mean_q: 3.088026
 42673/50000: episode: 787, duration: 0.544s, episode steps: 46, steps per second: 85, episode reward: 3.500, mean reward: 0.076 [-1.000, 0.100], mean action: -6.647 [-112.164, 24.557], mean observation: 0.040 [-5.886, 4.000], loss: 0.026549, mean_absolute_error: 0.107299, mean_q: 3.061661
 43043/50000: episode: 788, duration: 4.366s, episode steps: 370, steps per second: 85, episode reward: 35.900, mean reward: 0.097 [-1.000, 0.100], mean action: -0.744 [-98.096, 19.333], mean observation: 0.150 [-7.993, 4.000], loss: 0.023993, mean_absolute_error: 0.102078, mean_q: 3.111101
 43072/50000: episode: 789, duration: 0.344s, episode steps: 29, steps per second: 84, episode reward: 1.800, mean reward: 0.

NameError: name 'ENV_NAME' is not defined

In [10]:
# Finally, evaluate our algorithm for 5 episodes.
agent.test(env, nb_episodes=1000, visualize=True, nb_max_episode_steps=1001)

Testing for 5 episodes ...
Episode 1: reward: 20.000, steps: 200
Episode 2: reward: 20.000, steps: 200
Episode 3: reward: 20.000, steps: 200
Episode 4: reward: 20.000, steps: 200
Episode 5: reward: 20.000, steps: 200


In [11]:
# After training is done, we save the final weights.
ENV_NAME = '3DBall_128'
agent.save_weights('ddpg_{}_vec_weights.h5f'.format(ENV_NAME), overwrite=True)